# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(592, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,26.37,92,100,25.17,MP,1696945238
1,1,georgetown,5.4112,100.3354,28.86,87,20,4.12,MY,1696945107
2,2,cole harbour,44.6724,-63.4751,14.28,83,20,5.66,CA,1696945234
3,3,port-aux-francais,-49.3500,70.2167,1.31,68,99,18.45,TF,1696945239
4,4,mollendo,-17.0231,-72.0147,19.06,77,89,3.66,PE,1696945239


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles="OSM", 
                           title="World Map with discovered Cities",
                           xlabel='Latitudes', ylabel='Longitudes',
                           s='Humidity', c='City')

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Personal Ideal Conditions:- Max Temp <= 25, Cloudiness==0, hummidity<80
city_data_df=city_data_df[(city_data_df['Max Temp']<=25) & (city_data_df['Humidity']<80) & (city_data_df['Cloudiness']==0)] 

# Drop any rows with null values
city_data=city_data_df.dropna(axis=0, how='any')

# Display sample data
print(city_data.shape)
city_data.head()

(32, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,jiutai,44.1525,125.8328,5.19,43,0,1.80,CN,1696945239
82,82,naryn,41.4287,75.9911,9.94,27,0,1.31,KG,1696945250
107,107,hermanus,-34.4187,19.2345,20.29,63,0,9.03,ZA,1696945254
111,111,hamilton,39.1834,-84.5333,10.70,79,0,1.34,US,1696945255
157,157,urangan,-25.3000,152.9000,19.66,68,0,3.24,AU,1696945263


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

#Removing columns not needed
hotel_df = hotel_df.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date']) 
#Reorganizing the columns in the desired order
col_order=['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df=hotel_df[col_order]
#Adding new empty column
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,jiutai,CN,44.1525,125.8328,43,
82,naryn,KG,41.4287,75.9911,27,
107,hermanus,ZA,-34.4187,19.2345,63,
111,hamilton,US,39.1834,-84.5333,79,
157,urangan,AU,-25.3000,152.9000,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
# conditions = "wheelchair" 
# [THOUGHT OF ADDING THIS EARLIER BUT ALREADY GETTING FEW HOTELS, SO NOT RESTRICTING ANY FURTHER]

radius = 10000
limit = 5 

# Set the parameters for the type of search
params = {
   "categories":categories,
#     "conditions":conditions,
   "limit":limit,
   "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - ",
              f"nearest hotel: {hotel_df.loc[index, 'Hotel Name']} ")

# Display sample data
hotel_df.head()

Starting hotel search
jiutai -  nearest hotel: No hotel found 
levuka -  nearest hotel: No hotel found 
newman -  nearest hotel: No hotel found 
jining -  nearest hotel: No hotel found 
crane -  nearest hotel: No hotel found 
preobrazheniye -  nearest hotel: No hotel found 
turgen -  nearest hotel: No hotel found 


,City,Country,Lat,Lng,Humidity,Hotel Name
5,jiutai,CN,44.1525,125.8328,43,No hotel found
82,naryn,KG,41.4287,75.9911,27,Ала-Тоо
107,hermanus,ZA,-34.4187,19.2345,63,Aloe guest house
111,hamilton,US,39.1834,-84.5333,79,North Vista Manor
157,urangan,AU,-25.3000,152.9000,68,Ramada


In [20]:
not_fnd_series= hotel_df['Hotel Name']=='No hotel found'

In [21]:
not_fnd_df = hotel_df[not_fnd_series]
print(not_fnd_df.shape)
not_fnd_df.head(10)

(7, 6)


,City,Country,Lat,Lng,Humidity,Hotel Name
5,jiutai,CN,44.1525,125.8328,43,No hotel found
179,levuka,CA,50.1834,-105.4011,70,No hotel found
258,newman,US,37.3138,-121.0208,16,No hotel found
381,jining,CN,35.4050,116.5814,49,No hotel found
428,crane,US,31.3974,-102.3501,75,No hotel found
450,preobrazheniye,RU,42.9019,133.9064,59,No hotel found
500,turgen,KZ,43.4006,77.5933,35,No hotel found


In [22]:
fnd_hotel_df = hotel_df[~not_fnd_series]
print(fnd_hotel_df.shape)
fnd_hotel_df.head()

(25, 6)


,City,Country,Lat,Lng,Humidity,Hotel Name
82,naryn,KG,41.4287,75.9911,27,Ала-Тоо
107,hermanus,ZA,-34.4187,19.2345,63,Aloe guest house
111,hamilton,US,39.1834,-84.5333,79,North Vista Manor
157,urangan,AU,-25.3000,152.9000,68,Ramada
166,seminole,US,28.7086,-81.2081,73,Best Western Plus Sanford


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
hotel_fnd_map=fnd_hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles="OSM", 
                           xlim=(-180,180), ylim=(-60,55),
                           xlabel='Latitudes', ylabel='Longitudes', title="World Map with Hotels in hover message", 
                           s='Humidity', c='City', hover_cols=['Hotel Name', 'Country'])
# Display the map
hotel_fnd_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)